# MQTT Part 1: Subscribe and Publish



## MQTT Client for Python: Paho


Paho and is an [open-source Eclipse project](https://www.eclipse.org/paho/) that provides support for an MQTT client in Python. 

Just run the cell below to install the Python packages:

In [1]:
!python3 -m pip install --upgrade paho-mqtt

     ---------------------------------------- 99.4/99.4 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for paho-mqtt: started
  Running setup.py install for paho-mqtt: finished with status 'done'


  DEPRECATION: paho-mqtt is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: C:\Users\siver\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Read the output. If this command fails, use the pip command on your command line.

    python3 -m pip install --upgrade paho-mqtt

The [API docs of Paho are here](https://pypi.python.org/pypi/paho-mqtt).

We then need to import the package:

In [2]:
import paho.mqtt.client as mqtt

## MQTT Broker

For the communication via MQTT, you need an MQTT broker. 

* You can start your own MQTT broker so you have control over everything, but this may not work when you are on your local network (at home) and want to connect with others that should use the same broker.

* You can use the MQTT broker for this course, with the address `mqtt.item.ntnu.no`.

* For some testing and prototyping, you can also use some of the open MQTT brokers from various projects. Some of them are listed here: <https://github.com/mqtt/mqtt.github.io/wiki/public_brokers>

Of course, never send any sensitive information via these servers.

In [7]:
# broker, port = 'test.mosquitto.org', 1883
# broker, port = 'mqtt.flespi.io', 1883
# broker, port = 'mqtt.eclipse.org', 1883

# settings for a local MQTT broker on the same machine:
# broker, port = 'localhost', 1883
broker, port = "mqtt20.iik.ntnu.no", 1883

## Our Own MQTT Client in Python

We now want to create our own MQTT client. What we want to achieve is the following:

 * Connect to one of the public MQTT brokers.
 * Subscribe to the topic #.
 * List the incoming messages.
 * Disconnect after we received 20 messages.
 
The class below contains the code we need. We deciced to declare this as a Python class, since we want to have a variable `count` that counts the number of incoming messages, for then disconnecting after we received 20. Declaring a class gives us a good way to share the `count` variable. 

In [8]:
from threading import Thread


class MQTT_Client_1:
    def __init__(self):
        self.count = 0
        print(self.count)

    def on_connect(self, client, userdata, flags, rc):
        print("on_connect(): {}".format(mqtt.connack_string(rc)))

    def on_message(self, client, userdata, msg):
        print("on_message(): topic: {}".format(msg.topic))
        self.count = self.count + 1
        if self.count == 20:
            self.client.disconnect()
            print("disconnected after 20 messages")

    def start(self, broker, port):
        self.client = mqtt.Client()
        self.client.on_connect = self.on_connect
        self.client.on_message = self.on_message
        print("Connecting to {}:{}".format(broker, port))
        self.client.connect(broker, port)

        self.client.subscribe("#")
        try:
            thread = Thread(target=self.client.loop_forever)
            thread.start()
        except KeyboardInterrupt:
            print("Interrupted")
            self.client.disconnect()

Above we declare a class `MQTT_Client_1` that encapsulates the Paho MQTT client with our own logic. 

 * The method `__init__()` is called when we construct an instance of the class. We use it here to set our counter to 0. Note that we use `self.count` to declare this variable, so it is available also in the other methods.
 * The method `on_connect()` is called by the Paho client once a connection is established. We just print out a decoded form of the message we receive back to be sure about what is going on.
 * The method `on_message()` will be called by the Paho client every time we receive an MQTT message. Since we want to disconnect after the first 20 messages, we increase the counter in this method and also have the code for disconnection.
 * The method `start()` creates the client an connect to the broker. 
   * It also subscribes to the topic. 
   * In the end, it starts a thread that will run the client forever. 
   * We place this into a thread so that the notebook still stays responsive and prints on the received messages.
   * We also catch any KeyInterrupt error, in case you interrupt a notebook cell, so that we disconnect properly.

In [9]:
myclient = MQTT_Client_1()
myclient.start(broker, port)

0
Connecting to mqtt20.iik.ntnu.no:1883
on_connect(): Connection Accepted.


on_message(): topic: ttm4175
on_message(): topic: he
on_message(): topic: ttm4115
on_message(): topic: /team17
on_message(): topic: team17
on_message(): topic: group666
on_message(): topic: frank
on_message(): topic: tick
on_message(): topic: group4
on_message(): topic: team07
on_message(): topic: group666
on_message(): topic: tock
on_message(): topic: group666
on_message(): topic: tick
on_message(): topic: group666
on_message(): topic: tock
on_message(): topic: group666
on_message(): topic: tick
on_message(): topic: group666
on_message(): topic: tock
disconnected after 20 messages
